# PREDICTING HOUSE PRICES USING MULTIPLE LINEAR REGRESSION


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from math import sqrt, log, exp
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
%matplotlib inline 

In [2]:
train= pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
# adding SalePrice column to test dataset for consistency
test['SalePrice'] = 0
frames = [train, test]
df = pd.concat(frames)
df.shape

(2919, 81)

 ## Helper Functions

In [52]:
def missing_data(df):
    # Create a DataFrame of all features with the correspnding number of missing values     
    missing = df.isnull().sum().sort_values(ascending=False)
    missing_df = pd.DataFrame(missing, columns = ['Missing'])
    missing_table = missing_df[missing_df.Missing > 0]
    
    # Drop Features with more than 500 missing values
    df = df.drop(['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],axis=1)
    missing_table = missing_table.drop(['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],axis=0)
    
    # I found out that some numerical features are actually really categories, after going through kaggle user juliencs' notebook
    # https://www.kaggle.com/juliencs/a-study-on-regression-applied-to-the-ames-dataset

    df = df.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                     80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                     "MoSold"     : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                     7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}})
    
    # Convert the new categories feature to type category.  
    df['MSSubClass'] = df['MSSubClass'].astype('category')
    df['MoSold'] = df['MoSold'].astype('category')
    
    # Seperate the numerical missing features from non-numeric ones.    
    numeric = []
    non_numeric = []
    for missing in missing_table.index:
        if df[missing].dtype == np.object:
            non_numeric.append(missing)
        else:
            numeric.append(missing)
    
    # Fill all missing numeric features with O except for 'LotFrontage' feature which we replace with the mean.
    for feature in numeric:
        if feature != 'LotFrontage':
            df[feature] = df[feature].fillna(0)
        else:
            df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
        
    # Fill all missing non-numeric features with 'NA' and 'None' appropiately.     
    fill_with_no = ["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","GarageType","GarageFinish","GarageQual","GarageCond"]
    for feature in fill_with_no:
        df[feature] = df[feature].fillna("NA")
    
    df["MasVnrType"] = df["MasVnrType"].fillna("None")
    
    # No Absent Value for some features, so we'll go ahead to use the mode to replace all Nil values
    fill_with_mode = ["MSZoning","Utilities","Functional","Exterior1st","KitchenQual","Exterior2nd","SaleType","Electrical","MSSubClass","MoSold"]
    for feature in fill_with_mode:
        df[feature] = df[feature].fillna(df[feature].mode().iloc[0])
    
    # Final check for missing data
    if df.isnull().values.any():
        return 'There are still some missing values'
    else:
        return "There are no more missing values",df

def ordinal_categorical(df):
    # Converting Ordinal Variables into a numerical scale

    ordered_lotshape = ['IR3','IR2','IR1','Reg']
    df['LotShape'] = df['LotShape'].astype("category",ordered=True,categories=ordered_lotshape).cat.codes

    ordered_landcontour = ['Low','HLS','Bnk','Lvl']
    df['LandContour'] = df['LandContour'].astype("category",ordered=True,categories=ordered_landcontour).cat.codes

    ordered_exterqual = ['Po','Fa','TA','Gd','Ex']
    df['ExterQual'] = df['ExterQual'].astype("category",ordered=True,categories=ordered_exterqual).cat.codes

    ordered_extercond = ['Po','Fa','TA','Gd','Ex']
    df['ExterCond'] = df['ExterCond'].astype("category",ordered=True,categories=ordered_extercond).cat.codes

    ordered_bsmtqual = ['NA','Po','Fa','TA','Gd','Ex']
    df['BsmtQual'] = df['BsmtQual'].astype("category",ordered=True,categories=ordered_bsmtqual).cat.codes

    ordered_bsmtcond = ['NA','Po','Fa','TA','Gd','Ex']
    df['BsmtCond'] = df['BsmtCond'].astype("category",ordered=True,categories=ordered_bsmtcond).cat.codes

    ordered_bsmtexposure = ['NA','No','Mn','Av','Gd']
    df['BsmtExposure'] = df['BsmtExposure'].astype("category",ordered=True,categories=ordered_bsmtexposure).cat.codes

    ordered_bsmtfintype1 = ['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ']
    df['BsmtFinType1'] = df['BsmtFinType1'].astype("category",ordered=True,categories=ordered_bsmtfintype1).cat.codes

    ordered_bsmtfintype2 = ['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ']
    df['BsmtFinType2'] = df['BsmtFinType2'].astype("category",ordered=True,categories=ordered_bsmtfintype2).cat.codes

    ordered_heatingqc = ['Po','Fa','TA','Gd','Ex']
    df['HeatingQC'] = df['HeatingQC'].astype("category",ordered=True,categories=ordered_heatingqc).cat.codes

    ordered_kitchenqual = ['Po','Fa','TA','Gd','Ex']
    df['KitchenQual'] = df['KitchenQual'].astype("category",ordered=True,categories=ordered_kitchenqual).cat.codes

    ordered_functional = ['Sal','Sev','Maj1','Maj2','Mod','Min2','Min1','Typ']
    df['Functional'] = df['Functional'].astype("category",ordered=True,categories=ordered_functional).cat.codes

    ordered_garagefinish = ['NA','Unf','RFn','Fin']
    df['GarageFinish'] = df['GarageFinish'].astype("category",ordered=True,categories=ordered_garagefinish).cat.codes

    ordered_garagequal = ['NA','Po','Fa','TA','Gd','Ex']
    df['GarageQual'] = df['GarageQual'].astype("category",ordered=True,categories=ordered_garagequal).cat.codes

    ordered_garagecond = ['NA','Po','Fa','TA','Gd','Ex']
    df['GarageCond'] = df['GarageCond'].astype("category",ordered=True,categories=ordered_garagecond).cat.codes

    ordered_utilities = ['ELO','NoSeWa','NoSewr','AllPub']
    df['Utilities'] = df['Utilities'].astype("category",ordered=True,categories=ordered_utilities).cat.codes

    ordered_landslope = ['Sev','Mod','Gtl']
    df['LandSlope'] = df['LandSlope'].astype("category",ordered=True,categories=ordered_landslope).cat.codes

    ordered_electrical = ['Mix','FuseP','FuseF','FuseA','SBrkr']
    df['Electrical'] = df['Electrical'].astype("category",ordered=True,categories=ordered_electrical).cat.codes

    ordered_paveddrive = ['N','P','Y']
    df['PavedDrive'] = df['PavedDrive'].astype("category",ordered=True,categories=ordered_paveddrive).cat.codes
    
    ordinal = ['Utilities','LandSlope','Electrical','LotShape','PavedDrive','LandContour','ExterQual','GarageCond','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','KitchenQual','Functional','GarageFinish','GarageQual']
    ordinal_scale = {}
    for var in ordinal:
        ordinal_scale[var] = df[var].unique()
    
    return ordinal_scale

def log_rmse(model, X, y):
    '''
    The evaluation metric for this problem is Root-Mean-Squared-Error (RMSE) between the logarithm
    of the predicted value and the logarithm of the observed sales price. Since we already log transformed y
    earlier we compute the RMSE as it is.
    
    '''   
    # Fit model on X & y inp     
    model.fit(X,y)
    prediction = model.predict(X)
    
    return sqrt(mean_squared_error((y), prediction))

def generate_submission(test,model,submission_name):
    test_prediction = model.predict(test)
    test_pred = pd.DataFrame(abs(np.exp(test_prediction)))
    ID = pd.DataFrame(test['Id'])
    pred = ID.join(test_pred)
    submission_file = submission_name + '.csv'
    pred.to_csv(submission_file, header=['Id','SalePrice'],index=False)
    

 ## 1. Data Cleaning

 ### 1.1 Deal with missing data

In [4]:
# Run our predefined function that deals with missing data and returns a result variable & a clean dataset void of missing values. 
result, clean_data = missing_data(df)

# Print result of the missing_data function
print result

# Assign clean_data to df
df =  clean_data

There are no more missing values


 ### 1.2 Convert ordinal variables into a numerical scale

In [5]:
scale = ordinal_categorical(df)
scale

{'BsmtCond': array([3, 4, 0, 2, 1], dtype=int64),
 'BsmtExposure': array([1, 4, 2, 3, 0], dtype=int64),
 'BsmtFinType1': array([6, 5, 1, 3, 4, 0, 2], dtype=int64),
 'BsmtFinType2': array([1, 4, 0, 5, 3, 2, 6], dtype=int64),
 'BsmtQual': array([4, 3, 5, 0, 2], dtype=int64),
 'Electrical': array([4, 2, 3, 1, 0], dtype=int64),
 'ExterCond': array([2, 3, 1, 0, 4], dtype=int64),
 'ExterQual': array([3, 2, 4, 1], dtype=int64),
 'Functional': array([7, 6, 2, 5, 4, 3, 1], dtype=int64),
 'GarageCond': array([3, 2, 0, 4, 1, 5], dtype=int64),
 'GarageFinish': array([2, 1, 3, 0], dtype=int64),
 'GarageQual': array([3, 2, 4, 0, 5, 1], dtype=int64),
 'HeatingQC': array([4, 3, 2, 1, 0], dtype=int64),
 'KitchenQual': array([3, 2, 4, 1], dtype=int64),
 'LandContour': array([3, 2, 0, 1], dtype=int64),
 'LandSlope': array([2, 1, 0], dtype=int64),
 'LotShape': array([3, 2, 1, 0], dtype=int64),
 'PavedDrive': array([2, 0, 1], dtype=int64),
 'Utilities': array([3, 1], dtype=int64)}

## 2. Data Preprocessing

### 2.1 Outliers

In the Paper for this dataset the author pointed out a pitfall;


> Potential Pitfalls (Outliers): Although all known errors were corrected in the data, no observations have been removed due to unusual values and all final residential sales from the initial data set are included in the data presented with this article. There are five observations that an instructor may wish to remove from the data set before giving it to students (a plot of SALE PRICE versus GR LIV AREA will quickly indicate these points). Three of them are true outliers (Partial Sales that likely don’t represent actual market values) and two of them are simply unusual sales (very large houses priced relatively appropriately). **I would recommend removing any houses with more than 4000 square feet from the data set (which eliminates these five unusual observations) before assigning it to students.**  

So we go ahead to delete all observations with GrlivArea above 4000ft.

In [6]:
# From our combined we get back our train & test datasets
train = df[:1460]
test = df[1460:]

# Drop all rows in the train dataset with 'GrLivArea' greater than 4000
train = train.drop(train[train['GrLivArea'] >= 4000].index,axis=0)

# New shape of train data after removing outliers
print train.shape
print test.shape

# Combine train and test so we can continue performing other operations on the whole dataset
frames = [train, test]
df = pd.concat(frames)
print df.shape

# Make sure there is no null value in the target feature
np.where(np.isnan(df['SalePrice']))

(1456, 76)
(1459, 76)
(2915, 76)


(array([], dtype=int64),)

### 2.2 Standardization

If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected

In [7]:
# Get all numeric features
numerical_features = []
for feature in df.dtypes.index:
    # Exclude the target variable 'SalePrice' and 'Id' column    
    if (df[feature].dtype != np.object) and (str(df[feature].dtype) != 'category') and (feature != 'SalePrice') and (feature != 'Id') :
        numerical_features.append(feature)

scaler = StandardScaler()

# Again, from our combined df we get back our train & test datasets.
train = df[:1456]
test = df[1456:]

train.loc[:,numerical_features] = scaler.fit_transform(train[numerical_features])
test.loc[:,numerical_features] = scaler.transform(test[numerical_features])

# Combine train and test so we can continue performing other operations on the whole dataset
frames = [train, test]
df = pd.concat(frames)
train.head()


C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,SC60,RL,-0.221314,-0.202770,Pave,0.700717,0.3047,0.026216,Inside,...,-0.359882,-0.116501,-0.270606,-0.058115,-0.087809,Feb,0.137472,WD,Normal,208500
1,2,SC20,RL,0.496547,-0.086107,Pave,0.700717,0.3047,0.026216,FR2,...,-0.359882,-0.116501,-0.270606,-0.058115,-0.087809,May,-0.615009,WD,Normal,181500
2,3,SC60,RL,-0.077742,0.081281,Pave,-1.028509,0.3047,0.026216,Inside,...,-0.359882,-0.116501,-0.270606,-0.058115,-0.087809,Sep,0.137472,WD,Normal,223500
3,4,SC70,RL,-0.460601,-0.091179,Pave,-1.028509,0.3047,0.026216,Corner,...,4.086653,-0.116501,-0.270606,-0.058115,-0.087809,Feb,-1.367490,WD,Abnorml,140000
4,5,SC60,RL,0.687976,0.386636,Pave,-1.028509,0.3047,0.026216,FR2,...,-0.359882,-0.116501,-0.270606,-0.058115,-0.087809,Dec,0.137472,WD,Normal,250000


In [24]:
# Make sure there is no null value in the target feature
np.where(np.isnan(df['SalePrice']))

(array([], dtype=int64),)

### 2.3 One Hot Enconding

In [9]:
'''
Norminal Variables

MSZoning, Street, Alley, LotConfig, Neighborhood, Condition1, Condition2, BldgType, RoofStyle,
HouseStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,Heating, CentralAir, Electrical, GarageType
PavedDrive, Fence, MiscFeature, SaleType, SaleCondition, Utilities,LandSlope

'''
category = ['MSZoning', 'Street',  'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'RoofStyle','HouseStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating', 'CentralAir',  'GarageType', 'SaleType', 'SaleCondition','MSSubClass','MoSold']
df_dummies = pd.get_dummies(df[category])
df = df.drop(df[category],axis=1)
df = pd.concat([df, df_dummies],axis=1)

### 2.4 Log Transformation of target feature 'SalePrice'

In [10]:
# For the last time, from our combined df we get back our train & test datasets.
train = df[:1456]
test = df[1456:]

# Since it contains zero values which we put initially for consistency sake, and we are done with all transformations, we can now drop it.
test = test.drop(['SalePrice'],axis=1)

# Log transformation of the target feature
train.loc[:,'SalePriceLog'] = np.log(train['SalePrice'])

C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


## 3. MODEL SELECTION & TRAINING 


In [28]:
X = train.drop(['SalePrice','SalePriceLog'],axis=1)
y = train.SalePriceLog

### 3.1 Train Different Models

In [34]:
'''
The Dataset for this project was split into half; train and test, with the test set
not having the 'SalePrice' coulmn (target variable).
So we are left with the first half to train the model on and use cross validaition
to see how well each model performs.

'''
# Initialize different regression algorithms
lm = linear_model.LinearRegression()
Ridge = linear_model.Ridge()
decision_tree = DecisionTreeRegressor()
random_forest = RandomForestRegressor()
KNN = KNeighborsRegressor()

models = [lm, ridge, decision_tree, random_forest,KNN]
model_name = ['lm', 'ridge', 'decision_tree','random_forest','KNN']
scorer = make_scorer(r2_score)
result = {}

for name, model in enumerate(models):
    
    model.fit(X,y)
    train_score = model.score(X,y)
    cv_score = cross_val_score(model, X, y,cv=3, scoring = scorer)
    name = model_name[name]
    result[name] = [train_score,cv_score.mean()]

for model_scores in result:
    print model_scores, result[model_scores]

decision_tree [0.99999999448602372, 0.73322041865034138]
KNN [0.65392473773323534, -0.54371163726170602]
lm [0.94348033724576208, 0.89635697430574457]
ridge [0.9428013749682832, 0.90778012886287451]
random_forest [0.97705409127676335, 0.85361772460096363]


### 3.2 Model Selection and Hyperparameter tuning


| Scores       | Linear Regression | Ridge             | Decision Tree     | Random Forest     | KNN               |
|--------------|-------------------|-------------------|-------------------|-------------------|-------------------|
| Train Score  |    0.94348        |    0.94280        |    0.99999        |    0.97388        |    0.65392        |
| CV Score     |    0.89636        |    0.90778        |    0.72056        |    0.85411        |   -0.54371        |


In [57]:
ridge = linear_model.RidgeCV(alphas = [ 0.1, 10, 20, 30])
ridge.fit(X, y)
alpha = ridge.alpha_
print("Best alpha :", alpha)

print("Try again for more precision with alphas centered around " + str(alpha))
ridge = linear_model.RidgeCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
                          alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
                          alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4], cv = 10)
ridge.fit(X, y)
alpha = ridge.alpha_
print("Best alpha :", alpha)
train_prediction = ridge.predict(X)
print("Root Mean squared error (Log): %.5f" % log_rmse(ridge, X, y))
print(ridge.score(X, y))

('Best alpha :', 10.0)
Try again for more precision with alphas centered around 10.0
('Best alpha :', 11.5)
Root Mean squared error (Log): 0.09826
0.938419191579


In [56]:
generate_submission(test,ridge,'ridge_submission')